In [14]:
from WindPy import w
from datetime import datetime, timedelta
w.start()
date = '2022-11-18'
code = ['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF']
w.wsd(code[0], "sec_name,nav,NAV_adj_return1,return_1m,return_3m,return_1y", '2022-11-5', '2022-11-18', "annualized=1", usedf=True)[1]

,SEC_NAME,NAV,NAV_ADJ_RETURN1,RETURN_1M,RETURN_3M,RETURN_1Y
2022-11-07,平安短债A,1.1488,0.008705,2.492929,1.922233,3.170184
2022-11-08,平安短债A,1.1486,-0.017409,2.283034,1.710601,3.124439
2022-11-09,平安短债A,1.1485,-0.008706,2.178233,1.710751,3.124719
2022-11-10,平安短债A,1.1482,-0.026121,1.031175,1.570145,3.070018
2022-11-11,平安短债A,1.1478,-0.034837,0.514333,1.500236,3.034111
2022-11-14,平安短债A,1.1463,-0.130685,-1.325670,0.974999,2.890225
2022-11-15,平安短债A,1.1453,-0.087237,-2.334451,0.416772,2.791240
2022-11-16,平安短债A,1.1443,-0.087313,-3.333787,-0.034663,2.683058
2022-11-17,平安短债A,1.1436,-0.061173,-4.420734,-0.277055,2.601830
2022-11-18,平安短债A,1.1438,0.017489,-4.321913,-0.242434,2.601363


In [91]:
from typing import Literal
from WindPy import w
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

class Wind_Exporter:
    def __init__(self, code:str|list, indicator:str, method:Literal['wsd','wss']='wsd', Date_List:list=None, StartDate=None, EndDate=None, options=None):
        self.method = method
        self.code = code
        self.EndDate = EndDate if EndDate else datetime.today().strftime('%Y-%m-%d')
        self.Date_List = Date_List
        self.StartDate = EndDate if StartDate is None else StartDate
        self.indicator = indicator
        self.options = options
        self.data = []

    def check_connection(func):
        if not w.isconnected():
            w.start()
        return func

    @staticmethod
    def create_date_col(df, date):
        df['date'] = date
        column = np.roll(np.arange(len(df.columns)), 1)
        column[0], column[1] =column[1], column[0]
        return df.iloc[:, column]

    @check_connection
    def get_data(self):
        if self.method == 'wsd':
            return self.get_data_wsd()
        elif self.method == 'wss':
            return self.get_data_wss()
        else:
            raise ValueError('method must be wsd or wss')

    def get_data_wsd(self):
        if self.Date_List is None:
            if self.StartDate == self.EndDate:
                multi_data = []
                for i in self.code:
                    data_ = w.wsd(i, self.indicator, self.StartDate, self.EndDate, options=self.options, usedf=True)
                    multi_data.append(data_[1])
                df = self.create_date_col(pd.concat(multi_data, axis=0), self.EndDate) 
                self.data.append(df)
            else:
                multi_data = []
                for i in self.code:
                    data_ = w.wsd(i, self.indicator, self.StartDate, self.EndDate, options=self.options, usedf=True)
                    multi_data.append(data_[1])
                self.data.append(multi_data)
        else:
            for date in self.Date_List:
                multi_data = []
                for i in self.code:
                    data_ = w.wsd(i, self.indicator, date, date, options=self.options, usedf=True)
                    multi_data.append(data_[1])
                df = self.create_date_col(pd.concat(multi_data, axis=0), date)
                self.data.append(df)
        return self.data
    
    def get_data_wss(self):
        pass

indicator = "sec_name,nav,NAV_adj_return1,return_1m,return_3m,return_1y"
date = ['2022-11-18', '2022-11-17']
code = ['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF']
Wind_Exporter(code, method='wsd', indicator=indicator, EndDate='2022-11-18', options="annualized=1").get_data()
# wsd_parser(code, indicator=indicator, Date_List=date, options="annualized=1").get_data()





[           SEC_NAME        date     NAV  NAV_ADJ_RETURN1  RETURN_1M  \
 005754.OF     平安短债A  2022-11-18  1.1438         0.017489  -4.321913   
 005756.OF     平安短债E  2022-11-18  1.1314         0.017680  -4.566210   
 008911.OF  平安元丰中短债A  2022-11-18  1.0702         0.037390 -12.207985   
 008913.OF  平安元丰中短债E  2022-11-18  1.0633         0.037633 -12.377718   
 007935.OF   平安惠澜纯债A  2022-11-18  1.1266        -0.008875  -9.695035   
 007936.OF   平安惠澜纯债C  2022-11-18  1.1101        -0.009007 -10.209548   
 008696.OF  平安元盛超短债E  2022-11-18  1.0618         0.009419  -5.067381   
 004827.OF    平安中短债A  2022-11-18  1.1431         0.000000 -11.118021   
 006851.OF    平安中短债E  2022-11-18  1.1330         0.000000 -11.393370   
 
            RETURN_3M  RETURN_1Y  
 005754.OF  -0.242434   2.601363  
 005756.OF  -0.524266   2.342831  
 008911.OF  -3.196741   1.672050  
 008913.OF  -3.467365   1.430888  
 007935.OF  -2.768255   3.187397  
 007936.OF  -3.255709   2.672956  
 008696.OF  -1.554023   1.297462 

In [78]:
output = wsd_parser(code, indicator=indicator, Date_List=date, options="annualized=1").get_data()


TypeError: wsd_parser.create_date_col() takes 2 positional arguments but 3 were given

In [76]:
output[0]
wsd_parser.create_date_col(output[0], '2022-11-18')

,SEC_NAME,date,NAV,NAV_ADJ_RETURN1,RETURN_1M,RETURN_3M,RETURN_1Y
005754.OF,平安短债A,2022-11-18,1.1438,0.017489,-4.321913,-0.242434,2.601363
005756.OF,平安短债E,2022-11-18,1.1314,0.017680,-4.566210,-0.524266,2.342831
008911.OF,平安元丰中短债A,2022-11-18,1.0702,0.037390,-12.207985,-3.196741,1.672050
008913.OF,平安元丰中短债E,2022-11-18,1.0633,0.037633,-12.377718,-3.467365,1.430888
007935.OF,平安惠澜纯债A,2022-11-18,1.1266,-0.008875,-9.695035,-2.768255,3.187397
007936.OF,平安惠澜纯债C,2022-11-18,1.1101,-0.009007,-10.209548,-3.255709,2.672956
008696.OF,平安元盛超短债E,2022-11-18,1.0618,0.009419,-5.067381,-1.554023,1.297462
004827.OF,平安中短债A,2022-11-18,1.1431,0.000000,-11.118021,-3.463058,2.437494
006851.OF,平安中短债E,2022-11-18,1.1330,0.000000,-11.393370,-3.693938,2.173325


In [ ]:
pd.DateOffset